In [108]:
import pandas as pd
import numpy as np
import pickle
import nltk
import os
import joblib

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter
from textblob import TextBlob
from joblib import dump


<h3>Importamos el archivo para el procesamiento </h3>

In [109]:
df = pd.read_parquet(r'google_yelp.parquet',engine='pyarrow')

#### No tenemos que tener valores nulos por lo tanto hacemos un .dropna

In [111]:
df.dropna(subset='text',inplace=True)
df.dropna(subset='city',inplace=True)

# Tokenizacion y Lematizacion

In [112]:
df['tokens'] = df['text'].apply(lambda x: word_tokenize(x.lower())) 
lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
#tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords]
# Función de lematización
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords]
df['lemmatized_tokens'] = df['tokens'].apply(lemmatize_tokens)
#print(df['lemmatized_tokens'])
df['reconstructed_text'] = df['lemmatized_tokens'].apply(lambda tokens: ' '.join(tokens))


# Analisis de sentimiento

In [113]:
# Suponiendo que tienes un DataFrame llamado 'df' con una columna 'texto'
df['polaridad'] = df['reconstructed_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['sentimiento'] = df['polaridad'].apply(lambda x: 'positivo' if x > 0.3 else 'negativo' if x < 0 else 'negativo')
df['target'] = (df['sentimiento'] == 'positivo').astype(int)


# Vectorizador
#### Guardamos el vectorizador asi como la matriz de palabras resultantes para evitar volver a calcularlas luego.

In [114]:
# Entrenar el vectorizador con las reseñas
vectorizer = TfidfVectorizer(max_features=30000, stop_words='english')
vectorizer.fit(df['reconstructed_text'])

#2: Guardar la matriz resultante para evitar realizar esto en vivo en la aplicacion.
X = vectorizer.transform(df['reconstructed_text'])

# Regresion Logistica

In [115]:
y = df["target"]
# Divide el dataset en conjuntos de entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Importa el modelo de regresión logística de Scikit-learn
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = model.predict(X_test)
# Imprimir el reporte de clasificación
print("Reporte de Clasificación:")
print("--------------------------")
print(classification_report(y_test, y_pred))
# Imprimir la matriz de confusión
print("Matriz de Confusión:")
print("--------------------")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

Reporte de Clasificación:
--------------------------
              precision    recall  f1-score   support

           0       0.94      0.95      0.95    211030
           1       0.95      0.94      0.95    225294

    accuracy                           0.95    436324
   macro avg       0.95      0.95      0.95    436324
weighted avg       0.95      0.95      0.95    436324

Matriz de Confusión:
--------------------
[[200890  10140]
 [ 12787 212507]]


<h3>Guardamos los modelos en un archivo .pkl</h3>

In [116]:
with open('artefactos.pkl', 'wb') as archivo:
    pickle.dump({
        'model': model,
        'matriz': X,
        'vectorizer': vectorizer
    }, archivo)

### Hacemos las predicciones de nuestro modelo

# Filtros


- 1.0 Con el fin de evitar que la ciudad recomendada no contemple una variedad de categorias o una cantidad determinada de "negocios", se filtraron por aquellas ciudad que contengan mas de X valores unicos de locales.

- 1.1 Para evitar que las ciudades con mayor cantidad de reseñas tengan preponderancia sobre las de menor cantidad, se procedio a calcular un promedio en base a la similitud de los textos con la del usuario y filtrar la ciudad con mejor similaridad.

- 1.2 Filtro a traves del calculo del promedio de las estrellas por negocio, preparando la data para la recomendacion(negocios unicos) y tener un valor de la puntuacion(o satisfaccion de los usuarios) mas real o mas "general" del negocio y se mantuvieron las que posean un promedio mayor a X(3.5 o 4).

- 1.3 Filtro para las reseñas positivas.

- 1.4 Filtro para similaridad(similarity_score>0) debido a que todos los locales de una ciudad no poseen todos la misma similaridad y no es lo que busca el usuario.

- 1.5 Locales unicos para el usuario.


### Definimos una funcion de recomendacion para filtrar los resultados en la APP

In [117]:

def new_func():
    X = joblib.load('tfidf_matrix.pkl')
    return X

def similaridad(df, texto,vectorizador,matriz):
    new_text_vector = vectorizador.transform([texto])
    similarity_scores = cosine_similarity(new_text_vector, matriz)
    documentos_mas_similares = similarity_scores[0].argsort()[::-1] # indices de los valores mas similares
    # Filtrar índices fuera de rango
    documentos_mas_similares = [i for i in documentos_mas_similares if i < len(df)]
    # Solo seleccionamos los datos que tienen un índice válido
    datos_similares = df.iloc[documentos_mas_similares].copy()
    datos_similares.loc[:, 'similarity_score'] = similarity_scores[0][documentos_mas_similares]
    return datos_similares


#### Prueba de la funcion

In [119]:
nuevo_texto = 'i wanna take a walk in the night and then drink a beer in beatiful place'
df_sin_filtrar = similaridad(df=df,texto=nuevo_texto, vectorizador= vectorizer,matriz = X )
#df_sin_filtrar.drop(columns=['text','tokens', 'lemmatized_tokens', 'reconstructed_text', 'polaridad','sentimiento'],inplace=True)

In [120]:
def filtrar_datos(df_sin_filtrar):
    # 1.0: Filtrar ciudades con más de 100 negocios únicos
    unicos = df_sin_filtrar.groupby('city')['name'].nunique().reset_index()
    unicos.columns = ['ciudad', 'unicos']
    ciudades_filtradas = unicos[unicos['unicos'] > 100]['ciudad']
    df_filtrado = df_sin_filtrar[df_sin_filtrar['city'].isin(ciudades_filtradas)]

    # 1.1: Filtrar por la ciudad con el mayor puntaje de similitud promedio
    promedio_por_ciudad = df_filtrado.groupby('city')['similarity_score'].mean().reset_index()
    promedio_por_ciudad = promedio_por_ciudad.sort_values(by='similarity_score', ascending=False)
    ciudad_recomendada = promedio_por_ciudad.iloc[0].values[0]
    df_filtrado = df_filtrado[df_filtrado['city'] == ciudad_recomendada]

    # 1.2: Filtrar negocios con más de 3.5 estrellas promedio
    promedio_estrellas_negocio = df_filtrado.groupby('name')['stars_review'].mean().reset_index()
    promedio_estrellas_negocio = promedio_estrellas_negocio.query('stars_review > 3.5')
    df_final = df_filtrado.merge(promedio_estrellas_negocio, on='name', how='left', suffixes=('', '_promedio'))
    df_final.dropna(subset=['stars_review_promedio'], inplace=True)

    # 1.3: Filtrar por target = 1
    df_final = df_final[df_final['target'] == 1]

    # 1.4: Filtrar por similarity_score > 0
    df_final = df_final[df_final['similarity_score'] > 0]

    # 1.5: Eliminar duplicados por nombre de negocio
    df_final.drop_duplicates(subset='name', inplace=True)

    return df_final

In [122]:
df = filtrar_datos(df_sin_filtrar)

#### Mostramos el df filtrado con la similaridad del texto del usuario con los negocios

In [129]:
df = df[['name','city','similarity_score','stars_review_promedio']]
df

,name,city,similarity_score,stars_review_promedio
1,Royal House,New Orleans,0.311598,4.052632
2,Courtyard Brewery,New Orleans,0.296453,4.000000
4,Huge Ass Beers,New Orleans,0.261212,5.000000
6,Crescent City Brewhouse,New Orleans,0.237344,3.666667
7,Lost Love Lounge,New Orleans,0.233664,5.000000
...,...,...,...,...
1040,Martin Wine Cellar,New Orleans,0.011540,5.000000
1063,Magasin Vietnamese Cafe,New Orleans,0.010125,4.500000
1064,Seither's Seafood,New Orleans,0.010118,4.000000
1084,Bayona,New Orleans,0.008303,4.500000


#### Demostramos como funcionarian los artefactos

In [124]:
vec = joblib.load('artefactos.pkl')

In [125]:
vec['vectorizer']

TfidfVectorizer(max_features=30000, stop_words='english')

#### Observamos el Data Frame final con las columnas proporcionadas por los artefactos

In [130]:
df.head(3)

,name,city,similarity_score,stars_review_promedio
1,Royal House,New Orleans,0.311598,4.052632
2,Courtyard Brewery,New Orleans,0.296453,4.000000
4,Huge Ass Beers,New Orleans,0.261212,5.000000
